<a href="https://colab.research.google.com/github/CarineRam/Chatbot-ML-v2/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    with open(filename, 'r') as file:
        content = file.read()
        print(len(content))

Saving dialog.txt to dialog (2).txt
243904


In [48]:
chars = sorted(list(set(content)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"$&',-./0123456789:;?@abcdefghijklmnopqrstuvwxyzÂ—…
55


In [49]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[33, 34, 34, 2, 45, 33, 30, 43, 30]
hii there


In [50]:
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(content), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([243904]) torch.int64
tensor([33, 34,  8,  2, 33, 40, 48,  2, 26, 43, 30,  2, 50, 40, 46,  2, 29, 40,
        34, 39, 32, 24,  0, 34,  7, 38,  2, 31, 34, 39, 30, 10,  2, 33, 40, 48,
         2, 26, 27, 40, 46, 45,  2, 50, 40, 46, 43, 44, 30, 37, 31, 24,  1, 34,
         7, 38,  2, 31, 34, 39, 30, 10,  2, 33, 40, 48,  2, 26, 27, 40, 46, 45,
         2, 50, 40, 46, 43, 44, 30, 37, 31, 24,  0, 34,  7, 38,  2, 41, 43, 30,
        45, 45, 50,  2, 32, 40, 40, 29, 10,  2, 45, 33, 26, 39, 36, 44,  2, 31,
        40, 43,  2, 26, 44, 36, 34, 39, 32, 10,  1, 34,  7, 38,  2, 41, 43, 30,
        45, 45, 50,  2, 32, 40, 40, 29, 10,  2, 45, 33, 26, 39, 36, 44,  2, 31,
        40, 43,  2, 26, 44, 36, 34, 39, 32, 10,  0, 39, 40,  2, 41, 43, 40, 27,
        37, 30, 38, 10,  2, 44, 40,  2, 33, 40, 48,  2, 33, 26, 47, 30,  2, 50,
        40, 46,  2, 27, 30, 30, 39, 24,  1, 39, 40,  2, 41, 43, 40, 27, 37, 30,
        38, 10,  2, 44, 40,  2, 33, 40, 48,  2, 33, 26, 47, 30,  2, 50, 40, 46,
       

In [51]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [52]:
block_size = 8
train_data[:block_size+1]

tensor([33, 34,  8,  2, 33, 40, 48,  2, 26])

In [53]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([33]) the target: 34
when input is tensor([33, 34]) the target: 8
when input is tensor([33, 34,  8]) the target: 2
when input is tensor([33, 34,  8,  2]) the target: 33
when input is tensor([33, 34,  8,  2, 33]) the target: 40
when input is tensor([33, 34,  8,  2, 33, 40]) the target: 48
when input is tensor([33, 34,  8,  2, 33, 40, 48]) the target: 2
when input is tensor([33, 34,  8,  2, 33, 40, 48,  2]) the target: 26


In [54]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[50, 27, 30,  2, 31, 40, 46, 43],
        [ 2, 28, 40, 31, 31, 30, 30,  2],
        [30, 47, 30, 43,  2, 33, 46, 43],
        [44,  2, 34, 45,  2, 33, 34, 44]])
targets:
torch.Size([4, 8])
tensor([[27, 30,  2, 31, 40, 46, 43,  2],
        [28, 40, 31, 31, 30, 30,  2, 44],
        [47, 30, 43,  2, 33, 46, 43, 45],
        [ 2, 34, 45,  2, 33, 34, 44,  2]])
----
when input is [50] the target: 27
when input is [50, 27] the target: 30
when input is [50, 27, 30] the target: 2
when input is [50, 27, 30, 2] the target: 31
when input is [50, 27, 30, 2, 31] the target: 40
when input is [50, 27, 30, 2, 31, 40] the target: 46
when input is [50, 27, 30, 2, 31, 40, 46] the target: 43
when input is [50, 27, 30, 2, 31, 40, 46, 43] the target: 2
when input is [2] the target: 28
when input is [2, 28] the target: 40
when input is [2, 28, 40] the target: 31
when input is [2, 28, 40, 31] the target: 31
when input is [2, 28, 40, 31, 31] the target: 30
when input is [2, 28

In [55]:
print(xb) # our input to the transformer

tensor([[50, 27, 30,  2, 31, 40, 46, 43],
        [ 2, 28, 40, 31, 31, 30, 30,  2],
        [30, 47, 30, 43,  2, 33, 46, 43],
        [44,  2, 34, 45,  2, 33, 34, 44]])


In [56]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 55])
tensor(4.4512, grad_fn=<NllLossBackward0>)
	…&4
…m73-hag1cÂps5?9z8lt11,3
&jvqvzzkr:eos3
2vb m
Â9Âv/l?zyzv&!oed/-10$s	"n1/q….5p."01u'cp!9rl!:034w


In [57]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [58]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.248279333114624


In [59]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

	wane d bbe he t f dant d tof trop8 ice.	y oo, le thisol onoorulether.	t, ifone whyoithof bedingons ondout my theai ybsut ases d doritoou?
bes pest bu mps t se g ize ur t't noullobl?	wo tesbarorath&d iooul iche g ik wheeer tyoure me y?	$8:0;&he nt. heline t sh heaseee werk hadanuye ieav ne.

isopeallllargifoked he?
meavers yof ers ye cel t ctind ed at hed waveasoull, bacslias.	wicthenivet bube t twavendd cr t'moinikshave y ke, d ofi mu dis ino w, lor m ithe ds anthet tom..
in'r, anokilou.
widarat


Mathematical trick in self-attention

In [60]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [61]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [62]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [63]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [64]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [65]:
torch.manual_seed(42)
# a = torch.ones(3, 3)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [66]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])